In [1]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import metrics
import tensorflow.keras.preprocessing as tensor_image_gen
from sklearn.model_selection import train_test_split
import cv2
import os
from IPython.display import Image
import pandas as pd
import numpy as np
import cv2

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
img_train = tensor_image_gen.image_dataset_from_directory(
    'Sp_whole_copy\swimcat',
    labels='inferred',
    label_mode='categorical',
    class_names=('A-sky','B-pattern','C-thick-dark','D-thick-white','E-veil'),
    color_mode='rgb',
    batch_size=20,
    image_size=(125, 125),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
class Augment(tf.keras.layers.Layer):
  def __init__(self, seed=42):
    super().__init__()
    self.augment_inputs = tf.keras.Sequential([
            tf.keras.layers.RandomRotation(0.1, seed=seed),
            tf.keras.layers.RandomZoom(0.1, seed=seed),
            tf.keras.layers.RandomContrast(0.2, seed=seed),
            tf.keras.layers.RandomBrightness(0.5, seed=seed)
        ])
    self.augment_labels = tf.keras.Sequential([
            tf.keras.layers.RandomContrast(0.2, seed=seed),
            tf.keras.layers.RandomBrightness(0.2, seed=seed)
        ])

  def call(self, inputs, labels):
    inputs = self.augment_inputs(inputs)
    labels = self.augment_labels(labels)
    return inputs, labels

In [ ]:
img_train = img_train.map(Augment())

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                              patience=3)

In [ ]:
# Define the model
model = models.Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(125, 125, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(5, activation='sigmoid')
])

# Compile the model
model.compile(tf.keras.optimizers.SGD(learning_rate=0.001),loss='binary_crossentropy',metrics=[metrics.BinaryAccuracy()])

# Print the model summary
model.summary()


In [ ]:
history = model.fit(img_train, epochs=50, steps_per_epoch=None,callbacks=[callback])

In [ ]:
model.evaluate(img_train)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'])

plt.show()

In [ ]:
#model.save('sky_status_model2.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

model = load_model('my_model_10.h5')

In [ ]:
print(history.history['loss'])

In [ ]:
img_path = r'Cloud_copy\Cloud\cloud_augment_65.jpg'
try:
    img = image.load_img(img_path, target_size=(150, 150))
except FileNotFoundError:
    print(f"Error: The file {img_path} was not found.")
    exit()
    
img = np.array(img)

img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

In [ ]:
Image(img_path)

In [ ]:
model.predict(img_array)